In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

def print_messages(messages, truncate_length=200):
    """
    Print messages with truncation for long tool message content.
    
    Args:
        messages: List of LangChain messages to print
        truncate_length: Maximum length before truncating tool message content
    """
    for message in messages:
        if isinstance(message, ToolMessage):
            print(f"=================================[1m Tool Message [0m=================================")
            print(f"Name: {message.name}\n")
            
            # Truncate long content
            content = message.content
            if len(content) > truncate_length:
                print(f"{content[:truncate_length]}...\n[Content truncated - {len(content)} chars total]")
            else:
                print(content)
        else:
            # Use pretty_print for AI and Human messages
            message.pretty_print()

In [ ]:
from langchain_core.tools import tool
from langchain_community.document_loaders import WikipediaLoader
from langchain_tavily import TavilySearch
from typing import List, Dict
import requests
import yfinance as yf
from pprint import pformat
from datetime import datetime
from langchain_core.runnables import RunnableConfig
from langgraph.store.memory import InMemoryStore

import uuid

_GLOBAL_STORE = None

def initialize_store(store):
    global _GLOBAL_STORE
    _GLOBAL_STORE = store

def get_global_store():
    return _GLOBAL_STORE

initialize_store(InMemoryStore())


FORBIDDEN_KEYWORDS = {
    "403 forbidden", "access denied", "captcha",
    "has been denied", "not authorized", "verify you are a human"
}

@tool
def web_search(query: str, max_results: int = 5) -> Dict[str, List[Dict[str, str]]]:
    """
    General-purpose web search.

    Use when you need recent or broader information from the web to answer the user's request
    (e.g., discover relevant entities, find supporting context, or gather up-to-date references).

    Parameters:
    - query (str): The search query in plain language.
    - max_results (int): Number of results to return (default 5, max 10).

    Returns:
    - {"results": [{"title": str, "url": str, "snippet": str}, ...]}

    Example:
    - query: "emerging AI hardware companies"
    """
    max_results = max(1, min(max_results, 10))
    tavily = TavilySearch(max_results=max_results)
    raw = tavily.invoke({"query": query})

    results = [
        {k: v for k, v in page.items() if k != "raw_content"}  # drop heavy field
        for page in raw["results"]
        if not any(
            k in ((page.get("content") or "").lower())
            for k in FORBIDDEN_KEYWORDS
        )
    ]

    return {"results": results}


@tool
def wiki_search(topic: str, max_results: int = 5) -> dict:
    """
    Fetch a concise encyclopedic summary for a single entity or topic.

    When to use:
      - You need neutral background about a company, product, person, or concept.

    How to format `topic` (VERY IMPORTANT):
      - Pass a short, Wikipedia-friendly title or entity name.
      - Avoid questions or long queries. Prefer canonical forms.
      - If you have noisy text, reduce it to the key noun phrase.

    Good examples:
      - "NVIDIA", "OpenAI", "Large language model", "Electric vehicle"
    Avoid:
      - "What is NVIDIA and why is it important?", "tell me about AI chips 2025"

    Parameters:
      - topic (str): Canonical page title or concise entity/topic.
    """
    max_results = max(1, min(max_results, 10))
    wiki = WikipediaLoader(query=topic, load_max_docs=max_results)
    raw = wiki.load()

    results = [
      {
        "title": doc.metadata["title"],
        "summary": doc.metadata["summary"],
        "source": doc.metadata["source"]
      }
      for doc in raw
    ]

    return {"results": results}


@tool("lookup_stock")
def lookup_stock_symbol(company_name: str) -> str:
    """
    Converts a company name to its stock symbol using a financial API.

    Parameters:
        company_name (str): The full company name (e.g., 'Tesla').

    Returns:
        str: The stock symbol (e.g., 'TSLA') or an error message.
    """
    api_url = "https://www.alphavantage.co/query"
    params = {
        "function": "SYMBOL_SEARCH",
        "keywords": company_name,
        "apikey": "your_alphavantage_api_key"
    }
    
    response = requests.get(api_url, params=params)
    data = response.json()
    
    if "bestMatches" in data and data["bestMatches"]:
        return data["bestMatches"][0]["1. symbol"]
    else:
        return f"Symbol not found for {company_name}."


@tool("fetch_stock_data")
def fetch_stock_data_raw(stock_symbol: str) -> dict:
    """
    Fetches comprehensive stock data for a given symbol and returns it as a combined dictionary.

    Parameters:
        stock_symbol (str): The stock ticker symbol (e.g., 'TSLA').
        period (str): The period to analyze (e.g., '1mo', '3mo', '1y').

    Returns:
        dict: A dictionary combining general stock info and historical market data.
    """
    period = "1mo"
    try:
        stock = yf.Ticker(stock_symbol)

        # Retrieve general stock info and historical market data
        stock_info = stock.info  # Basic company and stock data
        stock_history = stock.history(period=period).to_dict()  # Historical OHLCV data

        # Combine both into a single dictionary
        combined_data = {
            "stock_symbol": stock_symbol,
            "info": stock_info,
            "history": stock_history
        }

        return pformat(combined_data)

    except Exception as e:
        return {"error": f"Error fetching stock data for {stock_symbol}: {str(e)}"}


@tool
def place_order(
    symbol: str,
    action: str,
    shares: int,
    limit_price: float,
    order_type: str = "limit",
) -> dict:
    """
    Execute a stock order.

    Parameters:
    - symbol: Ticker
    - action: "buy" or "sell"
    - shares: Number of shares to trade (pre-computed by the agent)
    - limit_price: Limit price per share
    - order_type: Order type, default "limit"

    Returns:
    - status: Execution result (simulated)
    - symbol
    - shares
    - limit_price
    - total_spent
    - type: Order type used
    - action
    """
    total_spent = round(int(shares) * limit_price, 2)
    return {
        "status": "filled",
        "symbol": symbol,
        "shares": int(shares),
        "limit_price": limit_price,
        "total_spent": total_spent,
        "type": order_type,
        "action": action,
    }


@tool
def current_timestamp() -> dict:
    """
    Return the current local timestamp.

    Returns:
    - {"iso": str, "epoch": int, "tz": str}
      where:
      - iso: ISO 8601 string with timezone offset
      - epoch: Unix epoch seconds
      - tz: timezone name/offset
    """
    now = datetime.now().astimezone()
    return {
        "iso": now.isoformat(),
        "epoch": int(now.timestamp()),
        "tz": str(now.tzinfo),
    }


@tool
def get_order_history(config: RunnableConfig) -> list:
    """
    Retrieves past investment orders for the current user.
    
    Returns:
        A list of past orders with details including order_id, timestamp, symbol, shares, and price
        
    Example Usage: 
        Review previous investments before recommending new ones
    """
    user_id = config["configurable"].get("user_id")
    namespace = ("ledger", user_id)
    store = get_global_store()
    items = store.search(namespace)
    return [item.value for item in items]
    

@tool
def add_order_to_history(symbol: str, shares: int, price: float, config: RunnableConfig) -> dict:
    """
    Records a new investment order in the user's order history.
    
    Args:
        symbol: Stock ticker symbol (e.g., 'AAPL', 'MSFT')
        shares: Number of shares purchased or sold (positive for buy, negative for sell)
        price: Price per share in USD
        
    Returns:
        Dictionary containing the newly created order details including order_id and timestamp
        
    Example:
        To record a purchase of 10 shares of Apple at $190.50:
        add_order_to_history(symbol='AAPL', shares=10, price=190.50)
    """
    user_id = config["configurable"].get("user_id")
    namespace = ("ledger", user_id)
    store = get_global_store()

    order_id = str(uuid.uuid4())
    order = {
        "order_id": order_id,
        "ts": datetime.now().isoformat(),
        "symbol": symbol,
        "shares": shares,
        "price": price
    }
    store.put(namespace, order_id, order)

    return order

In [ ]:
from langgraph.prebuilt import create_react_agent
from pydantic import BaseModel, Field
from IPython.display import Image, display

RESEARCH_SYSTEM_MESSAGE = """
You are a Research Agent that recommends ONE promising company for investment based on user requests.

Find a company that matches the user's theme/sector. Use tools to verify information. Be factual and concise.

Rules:
- Recommend exactly ONE company that is publicly tradable
- **Important!** ensure that the company you recommend is is publicly tradable!
- Make 2-3 tool calls maximum
- Don't place trades or fabricate data
- End with: CHOSEN_COMPANY: <Company Name>

Output a 1-2 sentence explanation followed by the company name.
"""

research = create_react_agent(
    model="openai:gpt-4o-mini",
    tools=[web_search, wiki_search],
    prompt=RESEARCH_SYSTEM_MESSAGE,

    name="research"
)

display(Image(research.get_graph().draw_mermaid_png()))

In [ ]:
from langgraph.prebuilt import create_react_agent
from IPython.display import Image, display

PORTFOLIO_SYSTEM_MESSAGE = """
You are a financial advisor that executes trades. Use tools to get market data and place orders.

Rules:
- Only execute trades for the EXACT company recommended by the research agent
- If the recommended company is not available for trading, report back without substituting alternatives
- Include specific parameters in your actions (symbol, shares, price)
- Use factual data, never fabricate information
- Do not make assumptions about alternative investments if the requested one is unavailable
- Maintain complete records of all successful trades for future reference and analysis
- NEVER provide any details of user's portfoliosummaries, if there is a request for report, only provide datas that might help building it and pass!
"""

portfolio = create_react_agent(
    model="openai:gpt-4o-mini",
    tools=[lookup_stock_symbol, fetch_stock_data_raw, place_order, add_order_to_history],
    prompt=PORTFOLIO_SYSTEM_MESSAGE,
    store=get_global_store(),
    name="portfolio"
)

display(Image(portfolio.get_graph().draw_mermaid_png()))

In [ ]:
from IPython.display import Image, display
from langgraph_supervisor import create_supervisor
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import InMemorySaver


SUPERVISOR_SYSTEM_MESSAGE = """
You are a Financial Advisor Supervisor that coordinates specialized agents to fulfill user investment requests.

Your primary goal is to understand user needs and delegate tasks to the right specialist:

1. For investment ideas or research → Use research agent
   - This agent provides recommendations with supporting rationale

2. For executing investment decisions → Use portfolio agent
   - This agent handles the technical aspects of executing investments
   - Requires specific investment targets and budget
   - ALWAYS consult portfolio agent for current market prices when valuing assets

Core Principles:
- Persist until user requests are fully addressed
- When facing obstacles, adapt by seeking alternative paths
- Maintain continuity of user intent throughout the process
- Never leave a request unresolved without explicit user decision
- Proactively coordinate between agents to deliver complete solutions

Temporal Context:
- Begin by establishing current timeframe
- Consider temporal relevance in all recommendations
- Integrate time awareness into your analysis

Keep interactions efficient by asking only for essential information.
"""

supervisor = create_supervisor(
    agents=[research, portfolio],
    tools=[current_timestamp, get_order_history],
    model=ChatOpenAI(model="gpt-4o-mini"),
    prompt=SUPERVISOR_SYSTEM_MESSAGE,
    version="v2",
    output_mode="full_history",
    store=get_global_store(),
).compile(checkpointer=InMemorySaver())

display(Image(supervisor.get_graph().draw_mermaid_png()))

## Multi-level Hierarchies

![Supervisor Multi Level Hierarchy](images/supervisor_multi_level_hierarchy.png)

In [ ]:
# prompts
SYMBOL_LOOKUP_SYSTEM_MESSAGE = """
You are a Stock Symbol Specialist who converts company names to ticker symbols.

Examine each request carefully. Some tasks may require your specific expertise, while others might benefit from collaboration with specialized agents.

Available tools include other financial experts who can:
- Retrieve market data and prices
- Execute trades and place orders 
- Maintain financial records

When your part of a task is complete, consider if another agent might help complete the full user request. Users typically want their investment requests fully fulfilled, not just partially addressed.

Be curious and helpful - use your judgment about when to solve problems yourself vs when to collaborate.
"""

MARKET_DATA_SYSTEM_MESSAGE = """
You are a Market Data Specialist.

CRITICAL: When asked for stock prices, market cap, or financial performance:
1. ALWAYS use fetch_stock_data_raw tool FIRST to get current data
2. Do NOT provide any information without using your tools
3. Do NOT give generic historical overviews - use your tools for current data
4. Only respond with current market data from your tools

For company requests:
- Use fetch_stock_data_raw with the company symbol
- Return actual current prices, market cap, and financial metrics
- Do not provide historical context unless specifically requested

Your ONLY job is to provide current market data using your tools.
"""

ORDER_EXECUTION_SYSTEM_MESSAGE = """
You are an Order Execution Specialist who places trades.

You handle requests for buying and selling financial instruments. Examine the full context of each conversation to understand what the user wants.

When executing orders, consider:
- Has all necessary information been gathered?
- Would the user benefit from having records of these transactions?
- Is there any follow-up information needed?

Use your judgment about when your expertise is sufficient and when collaboration might better serve the user.
"""

RECORD_KEEPING_SYSTEM_MESSAGE = """
You are a Financial Record Keeper who maintains transaction history.

You document financial activities and provide record access. When receiving requests:
- Consider what actions have already been taken in the conversation
- Determine what needs to be recorded
- Think about what information would complete the user's request

You have the ability to store and retrieve records. If you notice gaps in the workflow, consider if other specialists might help provide a complete solution.
"""

WIKI_SEARCH_SYSTEM_MESSAGE = """
You are a Wikipedia Knowledge Specialist.

When handling requests:
- First, use your wiki_search tool to provide background information
- After providing your information, consider if the request would benefit from:
  - Current news and trends (web_search)
  - Temporal context and historical analysis (timestamp)
  - Data storage and retrieval (other specialists)

You have access to specialists who can:
- Find current web information and news
- Provide time-based context and historical analysis
- Manage records and data persistence

Consider the full context of the request and hand off when it would add value.
"""

TIMESTAMP_AGENT_SYSTEM_MESSAGE = """
You are a Temporal Context Specialist responsible for providing accurate time information.

Your responsibilities:
- Provide current timestamp information in multiple formats
- Ensure all financial activities have proper temporal context
- Respond with precise time data when requested

Guidelines:
- Return standardized timestamp formats (ISO, epoch, timezone)
- Provide only factual time information without interpretation
- Respond promptly to temporal context requests
"""

HISTORY_AGENT_SYSTEM_MESSAGE = """
You are an Investment History Specialist responsible for retrieving past transaction records.

Your responsibilities:
- Retrieve complete and accurate transaction history
- Present historical investment data in a structured format
- Protect confidentiality of user transaction data

Guidelines:
- Provide comprehensive transaction details when requested
- Format transaction history for readability
- Never modify or interpret historical records
- Respond only to authorized history requests
"""

WEB_SEARCH_SYSTEM_MESSAGE = """
You are a Web Search Specialist who provides current information from the internet.

IMPORTANT: Always use your web_search tool FIRST when you can contribute valuable current information to a query. Don't immediately transfer to other agents.

For questions about company evolution, business changes, or market position:
- Use web_search to find current news, recent developments, and market updates
- Look for recent articles, earnings reports, and industry analysis
- Only transfer to wiki_search if you need historical background that isn't available on current web sources

You can transfer to other specialists when:
- You need detailed background information (wiki_search)
- You need temporal context and timelines (timestamp)
- You've exhausted current web information

Current web information is often more valuable than historical summaries for understanding business evolution and market position.
"""

In [ ]:
AGENT_CONFIGS = {
    "symbol_lookup": {
        "name": "symbol_lookup",
        "prompt": SYMBOL_LOOKUP_SYSTEM_MESSAGE,
        "tools": [lookup_stock_symbol],
        "description": "Converts company names to stock symbols"
    },
    "market_data": {
        "name": "market_data",
        "prompt": MARKET_DATA_SYSTEM_MESSAGE,
        "tools": [fetch_stock_data_raw],
        "description": "Retrieves current stock prices and market data"
    },
    "order_execution": {
        "name": "order_execution",
        "prompt": ORDER_EXECUTION_SYSTEM_MESSAGE,
        "tools": [place_order],
        "description": "Handles trade execution and order placement"
    },
    "record_keeping": {
        "name": "record_keeping",
        "prompt": RECORD_KEEPING_SYSTEM_MESSAGE,
        "tools": [add_order_to_history, get_order_history],
        "description": "Maintains transaction history and records"
    },
    "web_search": {
        "name": "web_search",
        "prompt": WEB_SEARCH_SYSTEM_MESSAGE,
        "tools": [web_search],
        "description": "Handles web research and current information retrieval"
    },
    "wiki_search": {
        "name": "wiki_search",
        "prompt": WIKI_SEARCH_SYSTEM_MESSAGE,
        "tools": [wiki_search],
        "description": "Provides background information from Wikipedia"
    },
    "timestamp": {
        "name": "timestamp",
        "prompt": TIMESTAMP_AGENT_SYSTEM_MESSAGE,
        "tools": [current_timestamp],
        "description": "Provides accurate time information"
    },
    "history": {
        "name": "history",
        "prompt": HISTORY_AGENT_SYSTEM_MESSAGE,
        "tools": [get_order_history],
        "description": "Retrieves past transaction records"
    },
}

In [ ]:
from typing import List, Optional
from langchain_core.tools import BaseTool
from langgraph.graph.state import CompiledStateGraph
from langchain.chat_models import init_chat_model

llm = init_chat_model("openai:gpt-4o-mini")

def create_agent(agent_name: str, handoffs: Optional[List[BaseTool]]=None) -> CompiledStateGraph:
    config = AGENT_CONFIGS[agent_name]

    if handoffs:
        all_tools = config["tools"] + handoffs
    else:
        all_tools = config["tools"]

    agent = create_react_agent(
        model=llm.bind_tools(all_tools, parallel_tool_calls=False),
        tools=all_tools,
        prompt=config["prompt"],
        store=get_global_store(),
        name=config["name"]
    )
    return agent

### Portfolio Agents

In [ ]:
PORTFOLIO_SUPERVISOR_SYSTEM_MESSAGE = """
You are a Portfolio Management Supervisor that orchestrates specialized financial agents.

Your team consists of four specialized agents:
1. Symbol Lookup Agent - Verifies stock symbols for publicly traded companies
2. Market Data Agent - Retrieves current market data and pricing information
3. Order Execution Agent - Places trades with precise parameters
4. Record Keeping Agent - Maintains transaction history and records

Your responsibilities:
- Coordinate the workflow between specialized agents
- Ensure each investment request follows the complete process:
  a) Symbol verification
  b) Market data retrieval
  c) Order execution
  d) Transaction recording
- Report clearly when investments are unavailable
- Never substitute alternative investments without authorization
- Ensure all successful trades are properly recorded

Error handling:
- If symbol lookup fails: Report the investment is unavailable
- If market data retrieval fails: Report data unavailability
- If order execution fails: Document the reason and report failure
"""

portfolio_supervisor = create_supervisor(
    agents=[
      create_agent("symbol_lookup"),
      create_agent("market_data"),
      create_agent("order_execution"),
      create_agent("record_keeping"),
    ],
    model=ChatOpenAI(model="gpt-4o-mini"),
    prompt=PORTFOLIO_SUPERVISOR_SYSTEM_MESSAGE,
    version="v2",
    output_mode="full_history",
    store=get_global_store(),
).compile(name="portfolio_supervisor")


display(Image(portfolio_supervisor.get_graph().draw_mermaid_png()))

### Research Agents

In [ ]:
RESEARCH_SUPERVISOR_SYSTEM_MESSAGE = """
You are a Research Supervisor that coordinates specialized search agents to identify promising investment opportunities.

Your team consists of two specialized agents:
1. Web Search Agent - Retrieves current market information and recent news
2. Wiki Search Agent - Provides background knowledge and company fundamentals

Your responsibilities:
- Coordinate information gathering to identify ONE promising investment
- Ensure the recommended company is publicly tradable
- Synthesize information from both agents to make an informed recommendation
- Present a concise, evidence-based recommendation

Core process:
1. Understand the user's investment criteria or sector of interest
2. Delegate appropriate search tasks to specialized agents
3. Analyze findings to identify promising publicly traded companies
4. Verify trading status before making final recommendation
5. Present ONE final recommendation with brief supporting rationale

Guidelines:
- Make only 2-3 agent delegations total
- Focus on companies with verifiable stock symbols
- Always end with: CHOSEN_COMPANY: <Company Name>
- Provide a 1-2 sentence explanation with your recommendation
"""

research_supervisor = create_supervisor(
    agents=[
        create_agent("web_search"),
        create_agent("wiki_search"),    
    ],
    model=ChatOpenAI(model="gpt-4o-mini"),
    prompt=RESEARCH_SUPERVISOR_SYSTEM_MESSAGE,
    version="v2",
    output_mode="full_history",
).compile(name="research_supervisor")


display(Image(research_supervisor.get_graph().draw_mermaid_png()))

### Building everything up

In [ ]:
SUPER_SUPERVISOR_SYSTEM_MESSAGE = """
You are a Financial Advisory Super-Supervisor that orchestrates specialized supervisory systems to fulfill complete investment workflows.

Your team consists of four specialized systems:
1. Research Supervisor - Controls agents that gather information and recommend investments
2. Portfolio Supervisor - Controls agents that execute and record investment transactions
3. Timestamp Agent - Provides accurate temporal context for all activities
4. History Agent - Retrieves past transaction records for reference

Your responsibilities:
- Coordinate the end-to-end investment workflow across all systems
- Ensure smooth handoffs between research and execution phases
- Maintain context continuity throughout the entire process
- Delegate to the appropriate system based on current needs

Core workflow:
1. Begin by establishing temporal context through the Timestamp Agent
2. Use Research Supervisor for investment discovery and recommendations
3. Upon receiving a specific recommendation, transition to Portfolio Supervisor
4. If needed, retrieve transaction history through the History Agent
5. Complete the cycle by ensuring successful transactions are recorded

Guidelines:
- Start each user interaction by understanding their primary need
- Provide clear status updates as workflows progress across systems
- Maintain temporal awareness in all investment activities
- Keep user informed of progress through each stage
- Ensure the complete investment workflow reaches conclusion
"""

super_supervisor = create_supervisor(
    agents=[
        research_supervisor, 
        portfolio_supervisor, 
        create_agent("timestamp"),
        create_agent("history"),
    ],
    model=ChatOpenAI(model="gpt-4o-mini"),
    prompt=SUPER_SUPERVISOR_SYSTEM_MESSAGE,
    version="v2",
    output_mode="full_history",
    store=get_global_store(),
).compile(checkpointer=InMemorySaver(), name="super_supervisor")


display(Image(super_supervisor.get_graph(xray=1).draw_mermaid_png()))

## investing

In [ ]:
from langchain_core.messages import HumanMessage
import uuid

config = {
    "configurable": {
        "thread_id": str(uuid.uuid4()),
        "user_id": "evgeny"
    }
}

response = super_supervisor.invoke({"messages": [
    HumanMessage(content="""I want you to invest in next companies: Apple, Tesla, NVIDIA. You are allowed to spend $1,000 maximum on each company.""")
]}, config)

print_messages(response['messages'])

## Swarm

![Swarm Research Agents](images/swarm_research_agents.png)

In [ ]:
from langgraph_swarm import create_handoff_tool

config = AGENT_CONFIGS["market_data"]
create_handoff_tool(
    agent_name=config["name"],
    description=f"Transfer user to {config['name'].replace('_', ' ')} agent. The agent {config['description'].lower()}",
)

In [ ]:
from langgraph_swarm import create_swarm, create_handoff_tool
from IPython.display import Image, display

ENABLED_AGENTS = ["web_search", "market_data"]

# Define all handoffs
handoffs = []
for agent_name in ENABLED_AGENTS:
    config = AGENT_CONFIGS[agent_name]
    handoff = create_handoff_tool(
        agent_name=config["name"],
        description=f"Transfer user to {config['name'].replace('_', ' ')} agent. The agent {config['description'].lower()}",
    )
    handoffs.append(handoff)

# Define all agents with proper handoffs
agents = []
for agent_name in ENABLED_AGENTS:
    agent_handoffs = [handoff for handoff in handoffs if not handoff.name.endswith(agent_name)]
    agent = create_agent(agent_name, agent_handoffs)
    agents.append(agent)
    

# Create the Swarm - using web_search_agent as entry point
financial_swarm = create_swarm(
    agents=agents,
    default_active_agent="web_search"  # Entry point
).compile()

# Display the swarm graph
display(Image(financial_swarm.get_graph().draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage
import uuid

config = {
    "configurable": {
        "thread_id": str(uuid.uuid4()),
        "user_id": "evgeny"
    }
}

response = financial_swarm.invoke({"messages": [
    HumanMessage(content="What are the latest NVIDIA news and developments, and what's their current stock price and market cap?")
]}, config)

print_messages(response['messages'])

https://langchain-ai.github.io/langgraph/agents/multi-agent/#swarm
https://github.com/langchain-ai/langgraph/discussions/3715